In [16]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

# Set up the LM
turbo = dspy.OpenAI(model="gpt-3.5-turbo-instruct", max_tokens=250)
dspy.settings.configure(lm=turbo)

# Load math questions from the GSM8K dataset
gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]

100%|██████████| 1319/1319 [00:00<00:00, 30096.87it/s]


In [3]:
print(gsm8k_trainset)

[Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'}), Example({'question': "Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.", 'gold_reasoning': 'Up a mountain, Stephen covered 3/4*40000 = <<3/4*40000=30000>>30000 feet. Coming down, Stephen covered another 30000 feet, making the total distance covered in one round to be 30000+30000 = <<30000+30000=60000>>60000. Since Stephen made 10 round trips up and down the mountain, he covered 10*60000 = <<10*60000=600000>>60

In [4]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")

    def forward(self, question):
        return self.prog(question=question)

In [5]:
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer: we want to "bootstrap" (i.e., self-generate) 4-shot examples of our CoT program.
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)

# Optimize! Use the `gsm8k_metric` here. In general, the metric is going to tell the optimizer how well it's doing.
teleprompter = BootstrapFewShot(metric=gsm8k_metric, **config)
optimized_cot = teleprompter.compile(CoT(), trainset=gsm8k_trainset)

 50%|█████     | 5/10 [00:08<00:08,  1.70s/it]


In [6]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(
    devset=gsm8k_devset,
    metric=gsm8k_metric,
    num_threads=4,
    display_progress=True,
    display_table=0,
)

# Evaluate our `optimized_cot` program.
evaluate(optimized_cot)

Average Metric: 7 / 10  (70.0): 100%|██████████| 10/10 [00:04<00:00,  2.02it/s]


70.0

In [7]:
turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?
Reasoning: Let's think step by step in order to find Marion's score. We know that Ella got 4 incorrect answers, which means she got 36 correct answers out of 40. We also know that Marion got 6 more than half of Ella's score, which is 6 more than 36/2 = 18. Therefore, Marion's score is 18 + 6 = 24.
Answer: 24

---

Question: Bridget counted 14 shooting stars in the night sky. Reginald counted two fewer shooting stars than did Bridget, but Sam counted four more shooting stars than did Reginald. How many more shooting stars did Sam count in the night sky than was the average n

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?\nReasoning: Let's think step by step in order to find Marion's score. We know that Ella got 4 incorrect answers, which means she got 36 correct answers out of 40. We also know that Marion got 6 more than half of Ella's score, which is 6 more than 36/2 = 18. Therefore, Marion's score is 18 + 6 = 24.\nAnswer: 24\n\n---\n\nQuestion: Bridget counted 14 shooting stars in the night sky. Reginald counted two fewer shooting stars than did Bridget, but Sam counted four more shooting stars than did Reginald. How many more shooting stars did Sam count in the night sky 

In [8]:
class Emotion(dspy.Signature):
    """Classify emotion among sadness, joy, love, anger, fear, surprise."""

    sentence = dspy.InputField()
    sentiment = dspy.OutputField()


sentence = "i started feeling a little vulnerable when the giant spotlight started blinding me"  # from dair-ai/emotion

classify = dspy.Predict(Emotion)
classify(sentence=sentence)

Prediction(
    sentiment='fear'
)

In [9]:
sentence = (
    "it's a charming and often affecting journey."  # example from the SST-2 dataset.
)

# 1) Declare with a signature.
classify = dspy.Predict("sentence -> sentiment")

# 2) Call with input argument(s).
response = classify(sentence=sentence)

# 3) Access the output.
print(response.sentiment)

Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: A third of the contestants at a singing competition are female, and the rest are male. If there are 18 contestants in total, how many of them are male?
Answer: 12

---

Question: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?
Answer: 296

---

Question: Bridget counted 14 shooting stars in the night sky. Reginald counted two fewer shooting stars than did Bridget, but Sam counted four more shooting stars than did Reginald. How many more shooting stars did Sam count in the night sky than was the average number of shooting stars observed for the three of them?
Ans

In [15]:
phi3 = dspy.OllamaLocal(model="phi3:latest")
orca = dspy.OllamaLocal(model="orca-mini:latest")
phi3("write a short poem")
orca("write a short poem")

[' In the stillness of the night,\nI hear the whispers of the stars,\nAs they twinkle and dance above,\nIn the sky that never seems to end.\n\nThe moon, a glowing orb of light,\nHangs serenely in the sky so high,\nCasting shadows upon the ground,\nAs creatures move and breathe with ease.\n\nThe air is cool and fresh and clean,\nAnd I feel my heart begin to sing,\nFor in this moment, all is well,\nAnd life is but a sweet dream.']

In [66]:
class CreateHaiku(dspy.Signature):
    """Create a one sentence haiku based on the topic."""

    topic = dspy.InputField()
    answer = dspy.OutputField(desc="a sentence with a haiku")


class HaikuPipeline(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.generate_haiku = dspy.Predict(CreateHaiku)

    def forward(self, topic):
        prediction = self.generate_haiku(topic=topic)
        return dspy.Prediction(topic=topic, answer=prediction.answer)


from dspy.teleprompt import BootstrapFewShot


# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    # answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_PM


# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

examples = [
    dspy.Example(
        topic="trees",
        answer="Golden leaves falling / Nature's dance with the wind / Serene and peaceful.",
    ),
    dspy.Example(
        topic="fun", answer="Joyful laughter / Fun bubbles up / Childhood memories."
    ),
    dspy.Example(
        topic="hope",
        answer="Hope blooms / Like a flower in spring / Brighten up our days.",
    ),
    # dspy.Example(topic="money", answer="Money piles high / A symbol of success / Negotiations begin."),
    # dspy.Example(topic="age", answer="Aging gracefully / Reflection of life / Nourishes growth."),
    # dspy.Example(topic="stupidity", answer="Stupidity never ends / Knowledge is power / Ignorance is not an option."),
    # dspy.Example(topic="poop", answer="Poop comes out of the pot / In the bathroom bush / Needs to go potty."),
    # dspy.Example(topic="nerds", answer="Nerds rule the world / Intelligence is power / Geek pride forever."),
    # dspy.Example(topic="bananas", answer="Bananas are delicious / Eating them makes me happy / Oh), the joy of fruit."),
    # dspy.Example(topic="whistles", answer="Whistling in the dark / Sounds so good in my ear / Peaceful and serene."),
    # dspy.Example(topic="beauty", answer="Beauty is in the eye of the beholder / A creation of nature / Timeless and evergreen."),
]

trainset = [x.with_inputs("topic") for x in examples]

# Compile!

In [65]:
compiled_haiku = teleprompter.compile(HaikuPipeline(), trainset=trainset)

 33%|███▎      | 1/3 [00:00<00:00, 550.94it/s]


AttributeError: 'Prediction' object has no attribute 'context'

In [61]:
my_topic = "castles"

pred = compiled_haiku(my_topic)

print(f"Question: {my_topic}")
print(f"Predicted Answer: {pred.answer}")

Question: castles
Predicted Answer: Majestic towers rise / A fortress of dreams and tales / A kingdom awaits.
